In [1]:
# 선형 회귀 모델
# 데이터셋 : https://www.data.go.kr/data/15007122/fileData.do
import pandas as pd

df = pd.read_csv("국민건강보험공단_건강검진정보_20221231.CSV", encoding='cp949')

pd.set_option('display.max_columns', None)

print(df.columns)
df.head()

Index(['기준년도', '가입자일련번호', '시도코드', '성별', '연령대코드(5세단위)', '신장(5cm단위)',
       '체중(5kg단위)', '허리둘레', '시력(좌)', '시력(우)', '청력(좌)', '청력(우)', '수축기혈압',
       '이완기혈압', '식전혈당(공복혈당)', '총콜레스테롤', '트리글리세라이드', 'HDL콜레스테롤', 'LDL콜레스테롤',
       '혈색소', '요단백', '혈청크레아티닌', '혈청지오티(AST)', '혈청지피티(ALT)', '감마지티피', '흡연상태',
       '음주여부', '구강검진수검여부', '치아우식증유무', '치석'],
      dtype='object')


,기준년도,가입자일련번호,시도코드,성별,연령대코드(5세단위),신장(5cm단위),체중(5kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,이완기혈압,식전혈당(공복혈당),총콜레스테롤,트리글리세라이드,HDL콜레스테롤,LDL콜레스테롤,혈색소,요단백,혈청크레아티닌,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부,구강검진수검여부,치아우식증유무,치석
0,2022,702763,41,2,14,160,75,89.0,1.0,1.0,1.0,1.0,127.0,73.0,84.0,NaN,NaN,NaN,NaN,14.8,1.0,0.8,25.0,20.0,15.0,1.0,0.0,0,NaN,NaN
1,2022,765435,41,2,9,150,45,68.0,1.0,1.0,1.0,1.0,127.0,75.0,85.0,NaN,NaN,NaN,NaN,13.1,1.0,0.8,19.0,10.0,7.0,1.0,1.0,0,NaN,NaN
2,2022,343621,41,2,13,150,60,78.8,1.0,1.0,1.0,1.0,135.0,81.0,105.0,NaN,NaN,NaN,NaN,13.0,1.0,0.8,17.0,15.0,15.0,1.0,1.0,0,NaN,NaN
3,2022,3393534,41,2,6,160,50,69.0,1.0,1.0,1.0,1.0,109.0,70.0,81.0,NaN,NaN,NaN,NaN,13.4,1.0,0.8,25.0,16.0,13.0,1.0,1.0,0,NaN,NaN
4,2022,1716951,41,2,7,160,55,75.0,1.0,1.0,1.0,1.0,110.0,67.0,79.0,NaN,NaN,NaN,NaN,14.7,1.0,0.8,22.0,22.0,15.0,1.0,1.0,0,NaN,NaN


In [2]:
# 데이터 전처리
# 시력, 청력, 치아 관련 데이터는 필요로 하지 않음
df.drop(["치아우식증유무", '치석', '시력(좌)', '시력(우)', '청력(좌)', '청력(우)', '구강검진수검여부'],
        axis=1, inplace=True)
# 중복값 제거
print("기준년도 칼럼 확인")
df.기준년도.value_counts()
# 고유값 제거
print("가입자일련번호 칼럼 확인")
df["가입자일련번호"].value_counts()
df.drop(["기준년도", "가입자일련번호", '성별', '시도코드'], axis=1, inplace=True)

test = df[df['LDL콜레스테롤'].isnull()]
train = df.dropna(axis=0)

print("학습 데이터 확인")
train.head(1)

기준년도 칼럼 확인
가입자일련번호 칼럼 확인
학습 데이터 확인


,연령대코드(5세단위),신장(5cm단위),체중(5kg단위),허리둘레,수축기혈압,이완기혈압,식전혈당(공복혈당),총콜레스테롤,트리글리세라이드,HDL콜레스테롤,LDL콜레스테롤,혈색소,요단백,혈청크레아티닌,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부
654336,13,165,60,80.0,118.0,68.0,103.0,243.0,120.0,85.0,134.0,14.0,1.0,1.0,22.0,17.0,34.0,2.0,1.0


In [3]:
y = train['LDL콜레스테롤']
x = train.drop('LDL콜레스테롤', axis=1)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

print("학습/검증 데이터 확인")
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

학습/검증 데이터 확인
(225877, 18) (225877,) (111253, 18) (111253,)


In [4]:
# 선형 회귀 학습
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(X_train, y_train)

print(f'''기울기 및 절편 확인
기울기 확인 coef = {reg.coef_}
절편 확인 intercept = {reg.intercept_}''')

print("전체에 대해서 가중치 확인")
for index, columns in enumerate(X_train.columns):
  print(f"{columns} = {reg.coef_[index]}")

기울기 및 절편 확인
기울기 확인 coef = [-0.00211547  0.00838286  0.00403974  0.02586561 -0.00362227 -0.00478919
 -0.00181933  0.98893442 -0.18998983 -0.90963871  0.04906444  0.04162509
  0.14300268 -0.00654572  0.00520954 -0.00195242  0.08355305 -0.22383297]
절편 확인 intercept = -7.701480658594193
전체에 대해서 가중치 확인
연령대코드(5세단위) = -0.0021154736576914114
신장(5cm단위) = 0.008382856140952466
체중(5kg단위) = 0.0040397441085129795
허리둘레 = 0.025865612399502663
수축기혈압 = -0.0036222690664538615
이완기혈압 = -0.004789193384670998
식전혈당(공복혈당) = -0.0018193256235440412
총콜레스테롤 = 0.9889344164631593
트리글리세라이드 = -0.189989828638398
HDL콜레스테롤 = -0.9096387125982945
혈색소 = 0.04906444369407519
요단백 = 0.041625090022681895
혈청크레아티닌 = 0.14300268237983307
혈청지오티(AST) = -0.006545718172645839
혈청지피티(ALT) = 0.005209544960069049
감마지티피 = -0.0019524161439546906
흡연상태 = 0.08355304678253973
음주여부 = -0.22383296735907016


In [5]:
# 예측
y_pred = reg.predict(X_test)

# 검증
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_pred, squared=False)  # RMSE
print(f'주요 Feature 삭제 전 rmse = {round(rmse, 3)}')

주요 Feature 삭제 전 rmse = 4.293


In [6]:
# 가중치가 큰 콜레스테롤 관련 Feature 삭제 후, 가중치 변화 관찰
x = x.drop(['총콜레스테롤', '트리글리세라이드', 'HDL콜레스테롤'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

lr = LinearRegression()
lr.fit(X_train, y_train)

print(f'''coef
      {lr.coef_}
      intercept
      {lr.intercept_}''')
print("가중치 비교")
for index, columns in enumerate(X_train.columns):
  print(f"{columns} = {reg.coef_[index]}")

y_pred = lr.predict(X_test)

rmse_2 = mean_squared_error(y_test, y_pred, squared=False)
print(f'주요 feature 삭제 후 rmse = {round(rmse_2, 3)}')

coef
      [-1.50293683 -0.39035452  0.15513633 -0.20859166 -0.20332664  0.40434892
 -0.13615129  3.43928937 -0.68370857 -2.45956598 -0.08639088  0.06739086
 -0.01048069 -2.77843294  0.15151187]
      intercept
      170.57752870857044
가중치 비교
연령대코드(5세단위) = -0.0021154736576914114
신장(5cm단위) = 0.008382856140952466
체중(5kg단위) = 0.0040397441085129795
허리둘레 = 0.025865612399502663
수축기혈압 = -0.0036222690664538615
이완기혈압 = -0.004789193384670998
식전혈당(공복혈당) = -0.0018193256235440412
혈색소 = 0.9889344164631593
요단백 = -0.189989828638398
혈청크레아티닌 = -0.9096387125982945
혈청지오티(AST) = 0.04906444369407519
혈청지피티(ALT) = 0.041625090022681895
감마지티피 = 0.14300268237983307
흡연상태 = -0.006545718172645839
음주여부 = 0.005209544960069049
주요 feature 삭제 후 rmse = 37.185
